# JrC Langchain

In [ ]:
import langchain
from langchain.cache import InMemoryCache
langchain.llm_cache = InMemoryCache()

## Chains

- Used to standardize the interface to functionality
- Thus, we can compose different chains together to achieve new functionality

In [ ]:
from __future__ import annotations

from typing import Any, Dict, List, Optional

from langchain.schema.language_model import BaseLanguageModel
from langchain.callbacks.manager import CallbackManagerForChainRun
from langchain.chains.base import Chain
from langchain.prompts import PromptTemplate


class LinkedInChain(Chain):
    prompt_template = PromptTemplate.from_template(
        """
        Generate a LinkedIn post that starts with "I'm proud and humble to..." and follows the typical format and tone of clichéd corporate announcements. Make the post about {activity} for a company named Junior Consulting.
        """
    )
    llm: BaseLanguageModel
    output_key: str = "post"

    @property
    def input_keys(self) -> List[str]:
        return self.prompt_template.input_variables

    @property
    def output_keys(self) -> List[str]:
        return [self.output_key]
    

    @property
    def _chain_type(self) -> str:
        return "linkedin_chain"

    def _call(
        self,
        inputs: Dict[str, Any],
        run_manager: Optional[CallbackManagerForChainRun] = None,
    ) -> Dict[str, str]:
        prompt_value = self.prompt_template.format_prompt(**inputs)
        response = self.llm.generate_prompt(
            [prompt_value], callbacks=run_manager.get_child() if run_manager else None
        )

        return {self.output_key: response.generations[0][0].text}


In [32]:
from langchain.callbacks.stdout import StdOutCallbackHandler
from langchain.chat_models.openai import ChatOpenAI

linkedin_chain = LinkedInChain(
    llm=ChatOpenAI(model="gpt-3.5-turbo"),
)

post = linkedin_chain.run({"activity": "successfully cooking breakfast"}, callbacks=[StdOutCallbackHandler()])
print("\n", post.replace("\n\n", "\n"))



> Entering new LinkedInChain chain...

> Finished chain.

 "I'm proud and humbled to announce that this morning, I had the absolute privilege of cooking breakfast for the brilliant minds at Junior Consulting. 
In the early hours, as the sun was rising, I found myself flipping pancakes, scrambling eggs, and brewing fresh coffee, all in the name of fostering team spirit and fueling productivity. It was an exercise in teamwork, communication, and the art of multitasking - skills that I apply daily in my professional life. 
I was reminded that it’s not just about the end product, but the journey you take to get there. The laughter, the camaraderie, the shared anticipation of a well-cooked meal – these are the moments that truly build a team. 
Thank you, Junior Consulting, for allowing me to serve you in a different capacity today. I am constantly inspired by your dedication, your passion, and your hunger for success (and pancakes).
Here's to more mornings filled with laughter, team spiri

## Agents

- To allow LLMs to to take actions
- Actions can be arbitrary, like calling APIs, reading files, etc.

### Tools

In [ ]:
from langchain.tools import Tool
from pydantic.v1 import BaseModel, Field
class LinkedInChainInput(BaseModel):
    activity: str = Field()

tools = []

class LinkedInChainInput(BaseModel):
    activity: str = Field()

tools.append(
    Tool.from_function(
        func=linkedin_chain.run,
        name="linkedin_post_generator",
        description="To generate a LinkedIn post about a specified activity",
        args_schema=LinkedInChainInput
    )
)

In [ ]:
import json
from Levenshtein import distance

class ConsultantInfoInput(BaseModel):
    name: str = Field(description="The name of the consultant you want information about")

def consultant_info(name: str) -> str:
    consultants = json.load(open("ansatte.json"))
    
    best_match_ld = float("inf")  
    best_match = None

    for consultant in consultants:
        consultant_name = consultant["first_name"]
        if name.count(" ") > 1:
            consultant_name += consultant["last_name"]

        ld = distance(name.lower(), consultant_name.lower())
        if ld < best_match_ld:
            best_match_ld = ld
            best_match = consultant

    return best_match

tools.append(
    Tool.from_function(
        func=consultant_info,
        name="consultant_info",
        description="To get info about a consultant",
        args_schema=ConsultantInfoInput
    )
)


### Junior Consulting Chatbot

In [33]:
from langchain.agents import AgentExecutor, OpenAIFunctionsAgent
from langchain.schema import SystemMessage
llm = ChatOpenAI(model="gpt-4")

prompt = OpenAIFunctionsAgent.create_prompt(
    system_message=SystemMessage(content="You are a chatbot that helps consultants in Junior Consulting do various tasks."),
)

agent = OpenAIFunctionsAgent(
    llm=llm, tools=tools, verbose=True, prompt=prompt
)
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True
)

In [35]:
question = "What is 2+2?"
answer = agent_executor.run(question)
print(answer)



> Entering new AgentExecutor chain...
The answer is 4.

> Finished chain.
The answer is 4.
